# Fraud Detection
----

In [2]:
import pandas as pd
import numpy as np
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from scipy import stats
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)
%matplotlib inline

In [53]:
df = pd.read_csv('./data/creditcard.csv')

In [54]:
print(df.shape)
df.head()

(284807, 31)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [55]:
X = df.loc[:, ~df.columns.isin(['Class'])]
Y = df['Class']

In [59]:
cols_to_norm = ['V1','V2','V3','V4','V5',
                'V6','V7','V8','V9','V10',
                'V11','V12','V13','V14',
                'V15','V16','V17','V18',
                'V19','V20','V21','V22',
                'V23','V24','V25','V26','V27','V28',]
X[cols_to_norm] = X[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

/Users/rook/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


## Random Forests

In [39]:
rfc = ensemble.RandomForestClassifier(n_estimators=5)

In [40]:
cvs = cross_val_score(rfc, X, Y, cv=10)

In [41]:
cvs

array([0.07081666, 0.99898181, 0.99975422, 0.99922756, 0.99933289,
       0.9988764 , 0.99961376, 0.99936798, 0.99940309, 0.99936798])

## Logistic Regression

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42, stratify=Y)

In [61]:
X_train['intercept'] = 1
X_test['intercept'] = 1

/Users/rook/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/rook/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [62]:
lr = LogisticRegression(C=1e5)

# Fit the model.
fit = lr.fit(X_train, y_train)

# Display.
print('Coefficients')
print(fit.coef_)
print(fit.intercept_)
pred_y_sklearn = lr.predict(X_test)

p_sklearn = np.where(pred_y_sklearn < .5, 0, 1)

print('\n Accuracy')
print(pd.crosstab(y_test, p_sklearn))

print('\n Percentage accuracy')
print(lr.score(X_test, y_test))

Coefficients
[[-2.47931505e-05 -3.51631087e-01 -2.72197002e-01 -3.21578673e-01
  -6.95181059e-02 -2.76916575e-01 -9.61689624e-02 -9.88122978e-02
  -2.80616928e-01 -1.83045464e-01 -1.95563484e-01 -8.97809550e-02
  -2.68725883e-01 -1.77502376e-01 -2.65300549e-01 -1.24079152e-01
  -1.76830913e-01 -2.87662694e-01 -2.50032040e-01 -1.94224361e-01
  -2.07489274e-01 -1.99513637e-01 -1.79291820e-01 -2.38333515e-01
  -1.39743123e-01 -2.09851002e-01 -1.52945008e-01 -1.49026112e-01
  -1.12028270e-01 -2.31464279e-04 -3.58344717e-01]]
[-0.35834472]

 Accuracy
col_0      0
Class       
0      71079
1        123

 Percentage accuracy
0.9982725204348193


## Feature Selection

In [35]:
X_kb = SelectKBest(chi2, k=15)
X_kb.fit_transform(X, Y)

array([[0.00000000e+00, 9.35192337e-01, 8.81364903e-01, ...,
        6.55065861e-01, 5.94863228e-01, 1.49620000e+02],
       [0.00000000e+00, 9.78541955e-01, 8.40298490e-01, ...,
        6.40681094e-01, 5.51930422e-01, 2.69000000e+00],
       [1.00000000e+00, 9.35217023e-01, 8.68140819e-01, ...,
        6.44945382e-01, 3.86683127e-01, 3.78660000e+02],
       ...,
       [1.72788000e+05, 9.90904812e-01, 7.81101998e-01, ...,
        6.80503729e-01, 5.18236397e-01, 6.78800000e+01],
       [1.72788000e+05, 9.54208999e-01, 8.49587129e-01, ...,
        7.29907941e-01, 7.89612050e-01, 1.00000000e+01],
       [1.72792000e+05, 9.49231759e-01, 8.49601462e-01, ...,
        6.64807343e-01, 5.43314293e-01, 2.17000000e+02]])

In [36]:
idxs_selected  = X_kb.get_support(indices=True)
X_new = X.columns[idxs_selected]
print(X_new)

Index(['Time', 'V1', 'V3', 'V4', 'V7', 'V9', 'V10', 'V11', 'V12', 'V14', 'V16',
       'V17', 'V18', 'V19', 'Amount'],
      dtype='object')
